In [ ]:
# ===================================================================
# 1. SETUP: Standard libraries
# ===================================================================
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json

# ===================================================================
# 2. LOAD PRE-TRAINED MODEL AND YOUR UPLOADED IMAGE
# ===================================================================
# Use a pre-trained ResNet-50 model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.eval()

# Hooks for Grad-CAM (this part remains the same)
final_conv_layer = model.layer4[2].conv3
gradients = None
activations = None
def backward_hook(module, grad_input, grad_output):
  global gradients
  gradients = grad_output[0]
def forward_hook(module, input, output):
  global activations
  activations = output
final_conv_layer.register_forward_hook(forward_hook)
final_conv_layer.register_backward_hook(backward_hook)

# Image preprocessing steps (this part remains the same)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Define the filename of the image you uploaded
image_filename = "/content/cat and dog.jpg"



try:
    # Open your local image file
    img = Image.open(image_filename).convert('RGB')
except FileNotFoundError:
    print(f"Error: The file '{image_filename}' was not found.")
    print("Please make sure you have uploaded the file and the filename is correct.")
    # Create a dummy image to prevent the rest of the code from crashing
    img = Image.new('RGB', (224, 224), color = 'red')

# Preprocess the image and add a "batch" dimension
input_tensor = preprocess(img).unsqueeze(0)


# ===================================================================
# 3. PERFORM CLASSIFICATION & GRAD-CAM (this part remains the same)
# ===================================================================
output = model(input_tensor)
pred_class_idx = output.argmax().item()

# Load ImageNet class labels
!wget -q https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open('imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]
print(f"Prediction: {labels[pred_class_idx]}")

output[:, pred_class_idx].backward()
pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
for i in range(activations.shape[1]):
    activations[:, i, :, :] *= pooled_gradients[i]
heatmap = torch.mean(activations, dim=1).squeeze().detach().numpy()
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

# ===================================================================
# 4. VISUALIZE THE RESULTS (this part remains the same)
# ===================================================================
resized_heatmap = cv2.resize(heatmap, (img.width, img.height))
colored_heatmap = plt.cm.jet(resized_heatmap)[:, :, :3]
original_img_array = np.float32(img) / 255
superimposed_img = (colored_heatmap * 0.4) + original_img_array
superimposed_img = np.clip(superimposed_img, 0, 1)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Original Image")
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(superimposed_img)
plt.title(f"Grad-CAM: Why '{labels[pred_class_idx]}'?")
plt.axis('off')
plt.show()